In [253]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

transaction_history_1 = pd.read_csv("../transaction_history_csv/sep-dec.csv")
transaction_history_2 = pd.read_csv("../transaction_history_csv/jun-sep.csv")
transaction_history = pd.concat([transaction_history_1, transaction_history_2])

# transaction_history = transaction_history[transaction_history["Reference"] == "UMC-S"]
transaction_history["Transaction Date"] = pd.to_datetime(transaction_history["Transaction Date"])
transaction_history["Debit Amount"] = transaction_history["Debit Amount"].replace({' ': np.nan})
transaction_history["Debit Amount"] = transaction_history["Debit Amount"].astype(float)
transaction_history["Credit Amount"] = transaction_history["Credit Amount"].replace({' ': np.nan})
transaction_history["Credit Amount"] = transaction_history["Credit Amount"].astype(float)
transaction_history["Date"] = transaction_history["Transaction Date"].dt.strftime('%b %d')
transaction_history["Month"] = transaction_history["Transaction Date"].dt.strftime('%m %b')
transaction_history["Transaction Ref1"] = transaction_history["Transaction Ref1"].fillna("")
transaction_history["Transaction Ref2"] = transaction_history["Transaction Ref2"].fillna("")
transaction_history["Transaction Ref3"] = transaction_history["Transaction Ref3"].fillna("")
transaction_history["Vendor"] = transaction_history["Transaction Ref1"] + ' ' + transaction_history["Transaction Ref2"] + ' ' + transaction_history["Transaction Ref3"]
transaction_history = transaction_history[transaction_history["Reference"] != "ITR"]

output_directory = "../transaction_history_csv/out.csv"

transaction_history

,Transaction Date,Reference,Debit Amount,Credit Amount,Transaction Ref1,Transaction Ref2,Transaction Ref3,Unnamed: 7,Date,Month,Vendor
0,2024-12-23,ICT,4.60,NaN,OTHR Na,PayNow Transfer 5114586,To: AL HIRA PTE. LTD.,NaN,Dec 23,12 Dec,OTHR Na PayNow Transfer 5114586 To: AL HIRA PT...
1,2024-12-21,ICT,500.00,NaN,OTHR PayNow Transfer,PayNow Transfer 8920040,To: CATALAN KWYNNZIE CHAVEZ,NaN,Dec 21,12 Dec,OTHR PayNow Transfer PayNow Transfer 8920040 T...
2,2024-12-21,ICT,NaN,485.0,OTHR OTHR,Incoming PayNow Ref 1590093,From: CATALAN KWYNNZIE CHAVEZ,NaN,Dec 21,12 Dec,OTHR OTHR Incoming PayNow Ref 1590093 From: CA...
3,2024-12-21,UMC-S,5.76,NaN,,SHENGSIONG@417FERNVALE SI NG 19DEC,5264-7110-0390-7302,NaN,Dec 21,12 Dec,SHENGSIONG@417FERNVALE SI NG 19DEC 5264-7110-...
4,2024-12-21,UMC-S,15.00,NaN,,KOPITIAM @VIVO SI NG 19DEC,5264-7110-0390-7302,NaN,Dec 21,12 Dec,KOPITIAM @VIVO SI NG 19DEC 5264-7110-...
...,...,...,...,...,...,...,...,...,...,...,...
126,2024-06-28,ICT,40.00,NaN,PayNow Transfer 9916061,To: SAMUEL SOON,OTHR PayNow Transfer,NaN,Jun 28,06 Jun,PayNow Transfer 9916061 To: SAMUEL SOON OTHR P...
127,2024-06-27,UMC-S,26.40,NaN,7-ELEVEN-COMPASS ONE SI NG 25JUN,5264-7110-1009-8095,,NaN,Jun 27,06 Jun,7-ELEVEN-COMPASS ONE SI NG 25JUN 5264-7110-1...
128,2024-06-27,UMC-S,3.55,NaN,GUARDIAN - COMPASS ONE SI NG 25JUN,5264-7110-1009-8095,,NaN,Jun 27,06 Jun,GUARDIAN - COMPASS ONE SI NG 25JUN 5264-7110-1...
129,2024-06-27,UMC-S,17.30,NaN,SENGKANG POLYCLINIC SI NG 25JUN,5264-7110-1009-8095,,NaN,Jun 27,06 Jun,SENGKANG POLYCLINIC SI NG 25JUN 5264-7110-1...


In [254]:
def remove_special_characters(string):
    return "".join(e for e in string.lower() if e.isalnum())

def check_if_string_in_list(original_string, reference_list):
    if not isinstance(original_string, str):
        return False
    original_list = [remove_special_characters(s) for s in re.split("\\s+|\.|\@|\*|\/|\_|\-", original_string)]
    return any([x in reference_list for x in original_list])

def categorize_vendors(original_string, reference_dict):
    if not isinstance(original_string, str):
        return "Others"
    original_list = [remove_special_characters(s) for s in re.split("\\s+|\.|\@|\*|\/|\_|\-", original_string)]
    for x in original_list:
        for key, value in reference_dict.items():
            if x in value:
                return key
    return "Shopping"

# F&B vendors list
with open("../restaurants.txt") as restaurants:
    lines = restaurants.readlines()

    restaurant_list = []
    for line in lines:
        line = [remove_special_characters(e) for e in line.split(", ")]
        restaurant_list.extend(line)

    restaurant_list = list(set(restaurant_list))

# Transport vendors list
# Transport, transfers, salary, and medical lists
transport_list = ["bus", "grab"]
# transfer_list = ["paynow", "transfer", "ref", "bank", "revolut"]
transfer_list = ["kwynnzie"]
salary_list = ["mindef", "saf", "gov"]
medical_list = ["polyclinic", "clinic"]

# Vendor group dictionary
vendor_groups = {
    "F&B": restaurant_list,
    "Transport": transport_list,
    "Transfers": transfer_list,
    "Salary": salary_list,
    "Medical": medical_list
}

# Transportation vendors list
transaction_history["Category"] = transaction_history["Vendor"].apply(lambda x: categorize_vendors(x, vendor_groups))
output_directory = "../transaction_history_csv/out.csv"
transaction_history["Category"].value_counts()

Category
Shopping     139
F&B           81
Transport     59
Transfers     49
Salary         7
Medical        1
Name: count, dtype: int64

In [255]:
def get_price_range(df, upper, lower=-np.inf):
    return df[(df["Debit Amount"] < upper) & (df["Debit Amount"] > lower)] 

def get_category(df, category):
    return df[df["Category"]==category]

def get_shopping_and_fb_categories(df):
    return df[(df["Debit Amount"].notna()) & (df["Category"].isin(["Shopping", "F&B"]))]

def output_csv(df, directory=output_directory):
    df.to_csv(directory, index=False)

def plot_catplot(df):
    df = get_shopping_and_fb_categories(df)

    month_axis_ordering = sorted(transaction_history["Month"].unique())
    sns.catplot(data=df, x="Month", y="Debit Amount", order=month_axis_ordering)

def plot_debit_amount(df):
    df[["Transaction Date", "Debit Amount"]].set_index("Transaction Date").plot.line()

def plot_monthly_debit_spending(df):
    df = df[df["Debit Amount"].notna()]
    ax = df[["Month", "Debit Amount"]].groupby("Month").sum().plot.bar()
    for p in ax.patches:
        ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

def plot_spending_histogram(df, bins_count=100):
    df["Debit Amount"].plot.hist(bins=bins_count)

def plot_number_of_transactions_monthly(df):
    df = df[df["Debit Amount"].notna()]
    ax = df[["Month", "Debit Amount"]].groupby("Month").count().plot.bar()
    for p in ax.patches:
        ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

In [256]:
matrix = transaction_history[transaction_history["Debit Amount"].notna()]
matrix = matrix[['Month', 'Category', 'Debit Amount']].groupby(['Month', "Category"]).agg(['mean', 'median', 'sum', 'count'])
matrix.columns = matrix.columns.droplevel()
matrix

mean   median      sum  count
Month  Category                                      
06 Jun F&B         20.987500   12.275    83.95      4
       Medical     17.300000   17.300    17.30      1
       Shopping    13.112500    6.775   104.90      8
       Transport    6.640000    6.640     6.64      1
07 Jul F&B         15.336250   13.150   122.69      8
       Shopping    17.468889    5.200   157.22      9
       Transfers    6.000000    6.000    12.00      2
       Transport   12.630000   12.630    25.26      2
08 Aug F&B         17.714286    9.000   124.00      7
       Shopping    22.250000   17.200   400.50     18
       Transfers   28.004000   20.000   140.02      5
       Transport   12.944000   11.540    64.72      5
09 Sep F&B         15.056875   14.350   240.91     16
       Shopping    22.748235   15.300   386.72     17
       Transfers   35.000000   35.000    70.00      2
       Transport   10.221667    6.330   122.66     12
10 Oct F&B         12.797333    8.150   191.96     15
       Shopping    26.594444   18.850   478.70     18
       Transfers   12.600000    5.000    63.00      5
       Transport   11.865833    8.350   142.39     12
11 Nov F&B         14.241154    9.300   370.27     26
       Shopping    31.235000   18.005   937.05     30
       Transfers  120.061538   50.000  1560.80     13
       Transport   18.824375   16.320   301.19     16
12 Dec F&B         10.160000   12.000    50.80      5
       Shopping    13.200000    9.000   435.60     33
       Transfers  300.000000  300.000   600.00      2
       Transport   14.125455   16.190   155.38     11

In [257]:
matrix = matrix["sum"].reset_index().pivot(index="Month", columns="Category", values="sum").drop("Medical", axis=1).fillna(0)
cols = matrix.columns
matrix

Category,F&B,Shopping,Transfers,Transport
Month,,,,
06 Jun,83.95,104.90,0.00,6.64
07 Jul,122.69,157.22,12.00,25.26
08 Aug,124.00,400.50,140.02,64.72
09 Sep,240.91,386.72,70.00,122.66
10 Oct,191.96,478.70,63.00,142.39
11 Nov,370.27,937.05,1560.80,301.19
12 Dec,50.80,435.60,600.00,155.38


In [258]:
def normalize(arr):
    return pd.Series([i/sum(arr) for i in arr])

matrix1 = matrix.apply(lambda x: normalize(x), axis=1)
matrix1.columns = cols
matrix1 = matrix1.reset_index()
matrix1

Category,Month,F&B,Shopping,Transfers,Transport
0,06 Jun,0.429434,0.536600,0.000000,0.033966
1,07 Jul,0.386827,0.495696,0.037835,0.079642
2,08 Aug,0.170040,0.549202,0.192008,0.088750
3,09 Sep,0.293689,0.471443,0.085336,0.149532
4,10 Oct,0.219120,0.546430,0.071914,0.162536
5,11 Nov,0.116830,0.295664,0.492473,0.095033
6,12 Dec,0.040909,0.350787,0.483177,0.125127


In [259]:
df = matrix.reset_index()

In [260]:
matrix1 = pd.melt(matrix1, value_vars=cols, id_vars=["Month"])

In [261]:
matrix1

,Month,Category,value
0,06 Jun,F&B,0.429434
1,07 Jul,F&B,0.386827
2,08 Aug,F&B,0.170040
3,09 Sep,F&B,0.293689
4,10 Oct,F&B,0.219120
5,11 Nov,F&B,0.116830
6,12 Dec,F&B,0.040909
7,06 Jun,Shopping,0.536600
8,07 Jul,Shopping,0.495696
9,08 Aug,Shopping,0.549202


In [262]:
def change_label(month):
    return sum(df[df["Month"]==month].values[0][1:])
matrix1["Month"] = matrix1["Month"].apply(change_label)

In [263]:
matrix1

,Month,Category,value
0,195.49,F&B,0.429434
1,317.17,F&B,0.386827
2,729.24,F&B,0.170040
3,820.29,F&B,0.293689
4,876.05,F&B,0.219120
5,3169.31,F&B,0.116830
6,1241.78,F&B,0.040909
7,195.49,Shopping,0.536600
8,317.17,Shopping,0.495696
9,729.24,Shopping,0.549202


In [264]:
import plotly.express as px

px.scatter(matrix1, y="value", x="Month", color="Category")